In [1]:
import pandas as pd

path1 = "./Folk.csv"
path2 = "./folk_songs_all_100.csv"
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

In [2]:

# check any of value in the track_name column in df2 is exist in the track)name column in df1
# remove those rows from df2
df2 = df2[~df2["track_name"].isin(df1["track_name"])]

# now check again
df2["track_name"].isin(df1["track_name"]).value_counts()

track_name
False    106
Name: count, dtype: int64

In [3]:
print(df2)

             artist_name              track_name  release_date genre
0          Woody Guthrie  This Land Is Your Land          1944  folk
1            Pete Seeger       If I Had a Hammer          1949  folk
2             Lead Belly        Goodnight, Irene          1933  folk
3       Elizabeth Cotten           Freight Train          1958  folk
4       Malvina Reynolds            Little Boxes          1962  folk
..                   ...                     ...           ...   ...
106               Hozier               Work Song          2014  folk
107        First Aid Kit                 Emmylou          2012  folk
108  Gregory Alan Isakov         The Stable Song          2007  folk
109       Brandi Carlile          That Wasn't Me          2012  folk
110      Phoebe Bridgers             Garden Song          2020  folk

[106 rows x 4 columns]


In [9]:
import lyricsgenius

# Replace "YOUR_CLIENT_ACCESS_TOKEN" with your actual Genius API client access token.
# genius = lyricsgenius.Genius("hx59ID6rCaaNkTEsf7nJDCdD4a2AmnK5N1MMllH8nJ3NSTu7p4tqD7lA9nH48ZBy")
# genius.headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
#     "Accept": "application/json",
#     "Authorization": f"Bearer {genius.access_token}"
# }

In [10]:
lyrics = []

songs = genius.search_songs('Begin Again Andy Shauf')
for song in songs['hits']:
    url = song['result']['url']
    song_lyrics = genius.lyrics(song_url=url)
    # id = song['result']['id']
    # song_lyrics = genius.lyrics(id)
    lyrics.append(song_lyrics)
    
print(lyrics)

HTTPError: [Errno 403] 403 Client Error: Forbidden for url: https://genius.com/Andy-shauf-begin-again-lyrics

In [11]:

genre = 'pop'

songs = []

page = 1
while page and len(songs) < 100:
    res = genius.tag(genre, page=page)
    print(res)
    for hit in res['hits']:
        # song_lyrics = genius.lyrics(song_url=hit['url'])
        print(hit['title'])

HTTPError: [Errno 403] 403 Client Error: Forbidden for url: https://genius.com/tags/pop/all?page=1

In [5]:
import time

def get_lyrics(track_name):
  attempts = 0
  max_attempts = 3  # Set a maximum number of attempts
  while attempts < max_attempts:
    try:
      song = genius.search_song(track_name)
      if song:
        print(f"Lyrics found for: {track_name}")  # Print success message
        return song.lyrics
      else:
        print(f"No lyrics found for: {track_name}")  # Print not found message
        return None
    except Exception as e:
      print(f"Error fetching lyrics for {track_name}: {e}")  # Print error message
      attempts += 1
      time.sleep(5)  # Wait for 5 seconds before retrying
  print(f"Failed to fetch lyrics for {track_name} after {max_attempts} attempts.")
  return None

In [6]:
df2["lyrics"] = df2["track_name"].apply(get_lyrics)

Searching for "This Land Is Your Land"...
Error fetching lyrics for This Land Is Your Land: [Errno 403] 403 Client Error: Forbidden for url: https://genius.com/api/search/multi?q=This+Land+Is+Your+Land
Searching for "This Land Is Your Land"...
Error fetching lyrics for This Land Is Your Land: [Errno 403] 403 Client Error: Forbidden for url: https://genius.com/api/search/multi?q=This+Land+Is+Your+Land
Searching for "This Land Is Your Land"...
Error fetching lyrics for This Land Is Your Land: [Errno 403] 403 Client Error: Forbidden for url: https://genius.com/api/search/multi?q=This+Land+Is+Your+Land
Failed to fetch lyrics for This Land Is Your Land after 3 attempts.
Searching for "If I Had a Hammer"...
Error fetching lyrics for If I Had a Hammer: [Errno 403] 403 Client Error: Forbidden for url: https://genius.com/api/search/multi?q=If+I+Had+a+Hammer
Searching for "If I Had a Hammer"...
Error fetching lyrics for If I Had a Hammer: [Errno 403] 403 Client Error: Forbidden for url: https://

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_lyrics_scrape(track_name, artist=None):
    query = f"{track_name} lyrics site:genius.com"
    search_url = f"https://www.google.com/search?q={query}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " +
                      "AppleWebKit/537.36 (KHTML, like Gecko) " +
                      "Chrome/120.0.0.0 Safari/537.36"
    }

    try:
        search_resp = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(search_resp.text, "html.parser")
        links = [a['href'] for a in soup.select("a") if "genius.com" in a['href']]
        if not links:
            print(f"No Genius link found for: {track_name}")
            return None

        genius_url = links[0].split("&")[0].replace("/url?q=", "")
        lyrics_page = requests.get(genius_url, headers=headers)
        soup = BeautifulSoup(lyrics_page.text, "html.parser")
        lyrics = soup.find("div", class_="lyrics")
        if not lyrics:
            lyrics = soup.find_all("div", class_=re.compile("^Lyrics__Container"))
            lyrics_text = "\n".join([l.get_text(separator="\n") for l in lyrics])
        else:
            lyrics_text = lyrics.get_text(separator="\n")

        print(f"Lyrics found for: {track_name}")
        return lyrics_text.strip()

    except Exception as e:
        print(f"Error fetching lyrics for {track_name}: {e}")
        return None


In [ ]:
df2["lyrics"] = df2["track_name"].apply(get_lyrics_scrape)


In [ ]:
pip install genius-lyrics


In [ ]:
from lyricsgenius import Genius
genius = Genius("hx59ID6rCaaNkTEsf7nJDCdD4a2AmnK5N1MMllH8nJ3NSTu7p4tqD7lA9nH48ZBy", skip_non_songs=True, remove_section_headers=True)

song = genius.search_song("This Land Is Your Land", "Woody Guthrie")
print(song.lyrics)


In [ ]:
from lyricsgenius import PublicAPI
song_id = 1  # the song you want tags for
public_api = PublicAPI()
song = public_api.song(song_id)['song']
print(song['tags'])